In [ ]:
import yfinance as yf
import plotly.express as px

In [20]:
df = yf.download("INFY.NS", period="6mo", multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Forward-filling missing data with previous day's data
df = df.ffill()

# Calculating daily percent returns
df["Returns"] = df["Close"].pct_change() * 100

# Calculating the 7 day rolling mean and standard deviation
df["Rolling_Returns"] = df["Returns"].rolling(7).mean()
df["Rolling_Volatility"] = df["Returns"].rolling(7).std()

# Upper and lower bounds for the Volatility Bands (Mean +/- Std Dev)
df["Upper"] = df["Rolling_Returns"] + df["Rolling_Volatility"]
df["Lower"] = df["Rolling_Returns"] - df["Rolling_Volatility"]

In [ ]:
# Plotting Daily Returns, Rolling Returns, and Volatility Bands
fig = px.line(
    df,
    x=df.index,
    y=["Returns", "Rolling_Returns", "Upper", "Lower"],
    width=1600,
    color_discrete_map={
        "Returns": "gray",
        "Rolling_Returns": "blue",
        "Upper": "rgba(0,0,0,0)",   # invisible line for upper
        "Lower": "rgba(0,0,0,0)"    # invisible line for lower
    },
    line_dash_map={
        "Returns": "dotted",
        "Rolling_Returns": "solid",
        "Upper": "solid",
        "Lower": "solid"
    },
    labels={
        "value": "Returns",
        "variable": "Legend"
    }
)

# Filling between Upper and Lower Bands using add_traces
fig.add_traces([
    dict(
        type="scatter",
        x=df.index,
        y=df["Upper"],
        mode="lines",
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    dict(
        type="scatter",
        x=df.index,
        y=df["Lower"],
        mode="lines",
        fill='tonexty',
        fillcolor='rgba(0,176,246,0.2)',  # light blue fill
        line=dict(width=0),
        name="Volatility Band",
        hoverinfo='skip'
    )
])

# Making Returns line less prominent
fig.update_traces(
    selector=dict(name="Returns"),
    line=dict(color="gray", width=1.5, dash="dot"),
    opacity=1
)

# Making Rolling Returns line more prominent
fig.update_traces(
    selector=dict(name="Rolling_Returns"),
    line=dict(color="blue", width=2)
)

fig.update_layout(
    title="Returns, Rolling Returns, and Volatility Band",
    yaxis_title="Returns",
    xaxis_title="Date"
)

fig